In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
taxonomy_list = gpd.read_file('../data/Taxonomy_List.csv')
taxonomy_list

,MEDICARE SPECIALTY CODE,MEDICARE PROVIDER/SUPPLIER TYPE DESCRIPTION,PROVIDER TAXONOMY CODE,PROVIDER TAXONOMY DESCRIPTION: TYPE CLASSIFICATION SPECIALIZATION,geometry
0,1,Physician/General Practice,208D00000X,Allopathic & Osteopathic Physicians/General Pr...,None
1,2,Physician/General Surgery,208600000X,Allopathic & Osteopathic Physicians/Surgery,None
2,2,Physician/General Surgery,2086H0002X,Allopathic & Osteopathic Physicians/Surgery/Ho...,None
3,2,Physician/General Surgery,2086S0120X,Allopathic & Osteopathic Physicians/Surgery/Pe...,None
4,2,Physician/General Surgery,2086S0122X,Allopathic & Osteopathic Physicians/Surgery/Pl...,None
...,...,...,...,...,...
550,D4,Undersea and Hyperbaric Medicine,207PE0005X,Allopathic & Osteopathic Physicians/Undersea a...,None
551,D5,Opioid Treatment Program,261QR0405X,Ambulatory Health Care Facilities/Clinic/Cente...,None
552,D6,Home Infusion Therapy Services,3336H0001X,Suppliers Pharmacy Home Infusion Therapy Services,None
553,D7,Micrographic Dermatologic Surgery,207ND0101X,Allopathic & Osteopathic Physicians Dermatolog...,None


In [3]:
data = gpd.read_file("/srv/data/my_shared_data_folder/pb-data/zcta.gpkg")
data.head()

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,STATEFP,TypeI,Geo_FIPS,Geo_QName,geometry
0,96799,96799,B5,G6350,S,197672934,1164505,-14.2231740,-169.5177430,11,3,NaN,96799,"MULTIPOLYGON (((-170.67152 -14.23885, -170.671..."
1,96770,96770,B5,G6350,S,177285639,33377439,+21.1336175,-157.2169319,11,3,96796770,96770,"MULTIPOLYGON (((-157.14885 21.09130, -157.1490..."
2,96769,96769,B5,G6350,S,193434451,44940998,+21.8958738,-160.1524561,11,3,96796769,96769,"MULTIPOLYGON (((-160.13518 21.94462, -160.1338..."
3,96752,96752,B5,G6350,S,10418464,1877589,+21.9709645,-159.7145071,11,3,96796752,96752,"MULTIPOLYGON (((-159.69767 21.95966, -159.6977..."
4,96796,96796,B5,G6350,S,119532951,11429247,+22.0940551,-159.7187487,11,3,96796796,96796,"MULTIPOLYGON (((-159.74093 22.10684, -159.7409..."


In [10]:
states = gpd.read_file("/srv/data/my_shared_data_folder/pb-data/states.gpkg")
states.head()

,REGION,DIVISION,Geo_FIPS,STATENS,GEOID,STUSPS,Geo_QNAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,9,0,60,01802701,60,AS,American Samoa,00,G4000,A,197759069,1307243751,-14.2671590,-170.6682674,"MULTIPOLYGON (((-168.14488 -14.54537, -168.144..."
1,4,9,15,01779782,15,HI,Hawaii,00,G4000,A,16634006436,11777792811,+19.5977643,-155.5024434,"MULTIPOLYGON (((-155.85049 20.26845, -155.8504..."
2,4,8,04,01779777,04,AZ,Arizona,00,G4000,A,294360282618,859561204,+34.2039355,-111.6063565,"MULTIPOLYGON (((-112.50381 31.78372, -112.5099..."
3,4,8,35,00897535,35,NM,New Mexico,00,G4000,A,314197253999,727781442,+34.4346843,-106.1316181,"MULTIPOLYGON (((-109.04894 31.96409, -109.0489..."
4,3,7,48,01779801,48,TX,Texas,00,G4000,A,676668210823,18991880422,+31.4347032,-099.2818238,"MULTIPOLYGON (((-101.25286 29.52050, -101.2535..."


In [ ]:
county = gpd.read_file("/srv/data/my_shared_data_folder/pb-data/county.gpkg")
county.head()